In [14]:
"""
https://www.kaggle.com/code/raghavkachroo/supreme-court-judgement-prediction/notebook
"""

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [15]:
df = pd.read_csv('justice.csv', delimiter=',', encoding = "utf8")
print(len(df))

# NaN값 삭제
df.dropna(inplace=True)
print(len(df))

df.head()

3303
3098


,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment
5,5,50644,Kleindienst v. Mandel,https://api.oyez.org/cases/1971/71-16,71-16,1971,"Richard G. Kleindienst, Attorney General of th...","Ernest E. Mandel, et al.",<p>Ernest E. Mandel was a Belgian professional...,2282,6,3,True,majority opinion,reversed,First Amendment


In [16]:
df1 = df.copy()

In [17]:
df1.drop(columns=['Unnamed: 0', 'docket','name','first_party','second_party', 'issue_area', 
                 'facts_len', 'majority_vote', 'minority_vote', 'href', 'ID','term'], inplace=True)

In [18]:
df1.head()

,facts,first_party_winner,decision_type,disposition
1,<p>Joan Stanley had three children with Peter ...,True,majority opinion,reversed/remanded
2,<p>John Giglio was convicted of passing forged...,True,majority opinion,reversed/remanded
3,"<p>The Idaho Probate Code specified that ""male...",True,majority opinion,reversed/remanded
4,"<p>Miller, after conducting a mass mailing cam...",True,majority opinion,vacated/remanded
5,<p>Ernest E. Mandel was a Belgian professional...,True,majority opinion,reversed


In [31]:
df_cat = df1[['decision_type', 'disposition']]
df_target = df1['first_party_winner']
df_nlp = df1['facts']
print(df_target)

0        True
1        True
2        True
3        True
4        True
        ...  
3093     True
3094     True
3095    False
3096     True
3097     True
Name: first_party_winner, Length: 3098, dtype: object


In [32]:
# NaN값 삭제
df_cat.reset_index(drop=True, inplace=True)
df_target.reset_index(drop=True, inplace=True)
df_nlp.reset_index(drop=True, inplace=True)

In [33]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
print(df_target)

# True and False -> 1, 0
df_target= label_encoder.fit_transform(df_target)
print(len(df_target))
print(df_target)

0        True
1        True
2        True
3        True
4        True
        ...  
3093     True
3094     True
3095    False
3096     True
3097     True
Name: first_party_winner, Length: 3098, dtype: object
3098
[1 1 1 ... 0 1 1]


In [34]:
df_target1 = pd.DataFrame(df_target, columns=['first_party_winner'])
df_target1

,first_party_winner
0,1
1,1
2,1
3,1
4,1
...,...
3093,1
3094,1
3095,0
3096,1


In [35]:
frames = [df_cat, df_target1]
df_concat = pd.concat(frames, axis=1, join='inner')
df_concat

,decision_type,disposition,first_party_winner
0,majority opinion,reversed/remanded,1
1,majority opinion,reversed/remanded,1
2,majority opinion,reversed/remanded,1
3,majority opinion,vacated/remanded,1
4,majority opinion,reversed,1
...,...,...,...
3093,majority opinion,reversed/remanded,1
3094,majority opinion,reversed/remanded,1
3095,majority opinion,affirmed,0
3096,majority opinion,vacated/remanded,1


In [39]:
# 불필요한 기호들 제거
df_nlp1 = pd.DataFrame(df_nlp, columns=['facts'])
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'<[^<>]*>', '', regex=True)
df_nlp1

,facts
0,Joan Stanley had three children with Peter Sta...
1,John Giglio was convicted of passing forged mo...
2,"The Idaho Probate Code specified that ""males m..."
3,"Miller, after conducting a mass mailing campai..."
4,Ernest E. Mandel was a Belgian professional jo...
...,...
3093,For over a century after the Alaska Purchase i...
3094,"Refugio Palomar-Santiago, a Mexican national, ..."
3095,Tarahrick Terry pleaded guilty to one count of...
3096,Joshua James Cooley was parked in his pickup t...


In [42]:
# 토큰화
corpus = df_nlp1["facts"]
lst_tokens = nltk.tokenize.word_tokenize(corpus.str.cat(sep=" "))

print(len(lst_tokens))
lst_tokens

611566


['Joan',
 'Stanley',
 'had',
 'three',
 'children',
 'with',
 'Peter',
 'Stanley',
 '.',
 'The',
 'Stanleys',
 'never',
 'married',
 ',',
 'but',
 'lived',
 'together',
 'off',
 'and',
 'on',
 'for',
 '18',
 'years',
 '.',
 'When',
 'Joan',
 'died',
 ',',
 'the',
 'State',
 'of',
 'Illinois',
 'took',
 'the',
 'children',
 '.',
 'Under',
 'Illinois',
 'law',
 ',',
 'unwed',
 'fathers',
 'were',
 'presumed',
 'unfit',
 'parents',
 'regardless',
 'of',
 'their',
 'actual',
 'fitness',
 'and',
 'their',
 'children',
 'became',
 'wards',
 'of',
 'the',
 'state',
 '.',
 'Peter',
 'appealed',
 'the',
 'decision',
 ',',
 'arguing',
 'that',
 'the',
 'Illinois',
 'law',
 'violated',
 'the',
 'Equal',
 'Protection',
 'Clause',
 'of',
 'the',
 'Fourteenth',
 'Amendment',
 'because',
 'unwed',
 'mothers',
 'were',
 'not',
 'deprived',
 'of',
 'their',
 'children',
 'without',
 'a',
 'showing',
 'that',
 'they',
 'were',
 'actually',
 'unfit',
 'parents',
 '.',
 'The',
 'Illinois',
 'Supreme',
 'C

In [ ]:
ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()